In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1862)

In [2]:
df = pd.read_excel("KHV_2015.xlsx", sheet_name = "KHV_2015", skiprows= 4)

In [3]:
df.head()

,Land,Kreis,Gemeinde,Name1,Name2,Name3,Name4,Straße/Hausnr,PLZ,Ort,Telefonvorwahl/-nummer,Telefaxvorwahl/-nummer,E-Mail Adresse,Internetadresse,Traeger,T_Name1,T_Name2,T_Name3,T_Name4,Art,Betten_Ins,Augen,Chirurgie,CH_Gefäßchir,CH_Thorax,CH_Unfall,CH_Viszeral,Frauen/Geburt,F-Frau,F_Geburt,HNO,Haut_Geschlecht,Herzchir,Herz_Thorax,Innere,I_Angio,I_Endo,I_Gastro,I_Haema,I_Kardio,I_Nephro,I_Pneumo,I_Rheuma,Geriatrie,Kinderchir,Kinderhk,K_Kardiologie,K_Neonatologie,Kinderpsychiat,Mund_Kiefer,Neurochir,Neurologie,Nuklear,Ortho,O_Rheuma,Plast_Chir,Psychiat,Psychiat_Sucht,Psychotherapeut,Strahlen,Urologie,Sonst,TN_Innere,TN_I_Geriatrie,TN_Kinderpsychiat,TN_Psychiat,TN_Psychotherapeut,TN_Sonst,TN_SUM,Änd_Art,Änd_Detail
0,1,1,0,Diakonissenkrankenhaus,NaN,NaN,NaN,Knuthstraße 1,24939,Flensburg,0461 8121-100,0461 8121-107,servicecenter@diako.de,www.diako.de,4,Ev.-luth. Diakonissenanstalt,zu Flensburg,NaN,NaN,2,531,1,101,20,-,81,-,66,-,-,-,-,-,-,107,-,-,-,-,-,-,-,-,-,-,33,-,-,-,-,44,43,-,-,-,8,79,-,8,-,38,3,-,-,25,48,15,-,88,NaN,NaN
1,1,1,0,Malteser Krankenhaus,St. Franziskus Hospital,NaN,NaN,Waldstraße 17,24939,Flensburg,0461 816-0,0461 816-2012,krankenhaus.flensburg@malteser.org,www.malteser-franziskus.de,4,Malteser Norddeutschland gGmbH,NaN,NaN,NaN,2,318,-,63,-,-,-,-,-,-,-,20,-,-,-,127,-,-,-,-,-,-,-,-,102,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,3,-,37,-,-,-,-,37,NaN,NaN
2,1,2,0,Augenklinik Bellevue,NaN,NaN,NaN,Lindenallee 21,24105,Kiel,0431 30108-0,0431 30108-40,uthoff@augenklinik-bellevue.de,www.augenklinik-bellevue.de,5,Prof. Dr. med. D. Uthoff,NaN,NaN,NaN,2,35,35,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN
3,1,2,0,Helios Klinik Kiel,NaN,NaN,NaN,Hamburger Chaussee 77,24113,Kiel,0431 7206-300,0431 7206-315,NaN,www.helios-kliniken.de/klinik/kiel,5,Helios Klinik Kiel GmbH,NaN,NaN,NaN,2,50,-,14,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,5,-,-,22,-,-,-,-,-,-,7,-,-,-,-,-,-,-,-,NaN,NaN
4,1,2,0,Klinik Flechsig,NaN,NaN,NaN,Chemnitzstr. 33,24116,Kiel,0431 3397100,0431 66940959,info@klinik-flechsig.de,www.klinik-flechsig.de,5,Klinik Flechsig GmbH & Co. KG,NaN,NaN,NaN,2,30,-,-,-,-,-,-,-,-,-,30,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN


In [4]:
df = df[['Land', 'Kreis', 'Gemeinde', 'Name1', 'Traeger', 'Betten_Ins', 'Straße/Hausnr', 'PLZ', 'Ort']]

In [5]:
df.sample(30)

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort
139,3,153,8,Privat-Nervenklinik,5,280,Lindenstraße 15,38704,Liebenburg
71,1,60,68,Psychiatrisches Krankenhaus,4,302,Daldorfer Strasse 2-6,24635,Rickling
112,2,0,0,Privita Privatklinik,5,1,Paul-Ehrlich-Straße 1,22457,Hamburg
1647,12,68,320,Ruppiner Kliniken GmbH,3,736,Fehrbelliner Strasse 38,16816,Neuruppin
1833,16,64,3,Hufeland Klinikum GmbH,3,551,Rudolph-Weiss-Straße 1-5,99947,Bad Langensalza
1575,11,5,5,Privatklinik Spandau PKS GmbH,5,6,Gatower Strasse 191,13595,Berlin
77,1,62,23,Park-Klinik Manhagen,5,212,Sieker Landstr.19,22927,Großhansdorf
972,8,212,0,Klinik am Stadtgarten,5,8,Beiertheimer Allee 18b,76137,Karlsruhe
534,5,562,24,LWL-Klinik Marl-Sinsen,1,119,Halterner Str. 525,45770,Marl-Sinsen
1649,12,69,17,Kliniken Beelitz GmbH,5,133,Paracelsusring 6a,14547,Beelitz-Heilstätten


## Limit dataset to NRW

In [6]:
# df = df.loc[lambda df: df['Land'] == 5]
df = df[df.Land == 5]

In [7]:
df.sample(30)

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort
499,5,382,68,GFO Kliniken Troisdorf,4,188,Wilhelm-Busch-Str. 9,53844,Troisdorf
551,5,570,52,Josephs-Hospital,4,261,Am Krankenhaus 2,48231,Warendorf
558,5,711,0,Krankenhaus Mara,4,177,Maraweg 21,33617,Bielefeld
433,5,315,0,Heilig Geist-Krankenhaus,4,283,Grasegger Str. 105,50737,Köln
568,5,766,20,Gemeindepsychiatrisches Zentrum,3,82,Schlabrendorffweg 2-6,32756,Detmold
364,5,116,0,Städt. Kliniken Mönchengladbach,3,555,Hubertusstr. 100,41239,Mönchengladbach
456,5,334,2,Universitätsklinikum Aachen,2,1456,Pauwelsstr. 30,52074,Aachen
567,5,766,8,Lippische Nervenklinik,5,116,Waldstr. 2,32105,Bad Salzuflen
386,5,154,36,Kath. Karl-Leisner-Klinikum,4,356,Albersallee 5-7,47533,Kleve
403,5,162,24,St.Alexius-/St.Josef-Krankenhaus,4,323,Nordkanalallee 99,41464,Neuss


## Limit dataset to one city to test API

In [8]:
# df = df.loc[lambda df: df['Ort'] == "Essen"]
# df = df[df.Ort == 'Essen']

In [9]:
# df.sample(10)

## Concatenate address

In [10]:
df = df.applymap(str)

In [11]:
df['address'] = df['Straße/Hausnr'].astype(str) + ', ' + \
                df['PLZ'] + ', ' + \
                df['Ort'] + ', ' + 'Germany'

In [12]:
df

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort,address
321,5,111,0,Dominikus-Krankenhaus,4,200,Am Heerdter Krankenhaus 2,40549,Düsseldorf,"Am Heerdter Krankenhaus 2, 40549, Düsseldorf, ..."
322,5,111,0,Ev. Krankenhaus,4,568,Kirchfeldstr. 40,40217,Düsseldorf,"Kirchfeldstr. 40, 40217, Düsseldorf, Germany"
323,5,111,0,Fliedner Klinik,4,-,Martin-Luther-Platz 26,40212,Düsseldorf,"Martin-Luther-Platz 26, 40212, Düsseldorf, Ger..."
324,5,111,0,Johanniter-Tagesklinik,4,-,Marbacher Str. 79,40597,Düsseldorf,"Marbacher Str. 79, 40597, Düsseldorf, Germany"
325,5,111,0,Krankenanstalten,4,558,Kreuzbergstr. 79,40489,Düsseldorf,"Kreuzbergstr. 79, 40489, Düsseldorf, Germany"
326,5,111,0,Krankenhaus Mörsenbroich-Rath,4,365,Amalienstr. 9,40472,Düsseldorf,"Amalienstr. 9, 40472, Düsseldorf, Germany"
327,5,111,0,LVR-Klinikum Düsseldorf,1,475,Bergische Landstr. 2,40629,Düsseldorf,"Bergische Landstr. 2, 40629, Düsseldorf, Germany"
328,5,111,0,Marien-Hospital,4,439,Rochusstr. 2,40479,Düsseldorf,"Rochusstr. 2, 40479, Düsseldorf, Germany"
329,5,111,0,Medical Center Düsseldorf,5,41,Hans-Günther-Sohl-Str. 6-10,40235,Düsseldorf,"Hans-Günther-Sohl-Str. 6-10, 40235, Düsseldorf..."
330,5,111,0,Paracelsus Klinik Golzheim,5,104,Friedrich-Lau-Str. 11,40474,Düsseldorf,"Friedrich-Lau-Str. 11, 40474, Düsseldorf, Germany"


## Get location and point

In [13]:
# locator = Nominatim(user_agent="myVeryFancyGeocoder")
# geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# df['location'] = df['address'].apply(geocode)
# df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [14]:
df.head()

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort,address,location,point
321,5,111,0,Dominikus-Krankenhaus,4,200,Am Heerdter Krankenhaus 2,40549,Düsseldorf,"Am Heerdter Krankenhaus 2, 40549, Düsseldorf, ...","(Schön Klinik Düsseldorf, 2, Am Heerdter Krank...","(51.23135345, 6.727251670737527, 0.0)"
322,5,111,0,Ev. Krankenhaus,4,568,Kirchfeldstr. 40,40217,Düsseldorf,"Kirchfeldstr. 40, 40217, Düsseldorf, Germany","(Evangelisches Krankenhaus, 40, Kirchfeldstraß...","(51.213210849999996, 6.773057660453232, 0.0)"
323,5,111,0,Fliedner Klinik,4,-,Martin-Luther-Platz 26,40212,Düsseldorf,"Martin-Luther-Platz 26, 40212, Düsseldorf, Ger...","(26, Martin-Luther-Platz, Stadtmitte, Stadtbez...","(51.2251325, 6.7817526, 0.0)"
324,5,111,0,Johanniter-Tagesklinik,4,-,Marbacher Str. 79,40597,Düsseldorf,"Marbacher Str. 79, 40597, Düsseldorf, Germany","(Johanniter-Tagesklinik Düsseldorf-Benrath, 79...","(51.1643955, 6.8639209, 0.0)"
325,5,111,0,Krankenanstalten,4,558,Kreuzbergstr. 79,40489,Düsseldorf,"Kreuzbergstr. 79, 40489, Düsseldorf, Germany","(Florence-Nightingale-Krankenhaus, 79, Kreuzbe...","(51.3003466, 6.7493957, 0.0)"


## Split point into lattitude, longitude and attitude

In [15]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [16]:
df.head()

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort,address,location,point,latitude,longitude,altitude
321,5,111,0,Dominikus-Krankenhaus,4,200,Am Heerdter Krankenhaus 2,40549,Düsseldorf,"Am Heerdter Krankenhaus 2, 40549, Düsseldorf, ...","(Schön Klinik Düsseldorf, 2, Am Heerdter Krank...","(51.23135345, 6.727251670737527, 0.0)",51.231353,6.727252,0.0
322,5,111,0,Ev. Krankenhaus,4,568,Kirchfeldstr. 40,40217,Düsseldorf,"Kirchfeldstr. 40, 40217, Düsseldorf, Germany","(Evangelisches Krankenhaus, 40, Kirchfeldstraß...","(51.213210849999996, 6.773057660453232, 0.0)",51.213211,6.773058,0.0
323,5,111,0,Fliedner Klinik,4,-,Martin-Luther-Platz 26,40212,Düsseldorf,"Martin-Luther-Platz 26, 40212, Düsseldorf, Ger...","(26, Martin-Luther-Platz, Stadtmitte, Stadtbez...","(51.2251325, 6.7817526, 0.0)",51.225133,6.781753,0.0
324,5,111,0,Johanniter-Tagesklinik,4,-,Marbacher Str. 79,40597,Düsseldorf,"Marbacher Str. 79, 40597, Düsseldorf, Germany","(Johanniter-Tagesklinik Düsseldorf-Benrath, 79...","(51.1643955, 6.8639209, 0.0)",51.164395,6.863921,0.0
325,5,111,0,Krankenanstalten,4,558,Kreuzbergstr. 79,40489,Düsseldorf,"Kreuzbergstr. 79, 40489, Düsseldorf, Germany","(Florence-Nightingale-Krankenhaus, 79, Kreuzbe...","(51.3003466, 6.7493957, 0.0)",51.300347,6.749396,0.0


## Clean up columns

In [17]:
df.columns

Index(['Land', 'Kreis', 'Gemeinde', 'Name1', 'Traeger', 'Betten_Ins',
       'Straße/Hausnr', 'PLZ', 'Ort', 'address', 'location', 'point',
       'latitude', 'longitude', 'altitude'],
      dtype='object')

In [ ]:
# df = df.drop(['Land', 'Kreis', 'Gemeinde', 'Straße/Hausnr','PLZ', 'Ort', 'address', 'location', 'point'], axis=1)

In [18]:
df.head()

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort,address,location,point,latitude,longitude,altitude
321,5,111,0,Dominikus-Krankenhaus,4,200,Am Heerdter Krankenhaus 2,40549,Düsseldorf,"Am Heerdter Krankenhaus 2, 40549, Düsseldorf, ...","(Schön Klinik Düsseldorf, 2, Am Heerdter Krank...","(51.23135345, 6.727251670737527, 0.0)",51.231353,6.727252,0.0
322,5,111,0,Ev. Krankenhaus,4,568,Kirchfeldstr. 40,40217,Düsseldorf,"Kirchfeldstr. 40, 40217, Düsseldorf, Germany","(Evangelisches Krankenhaus, 40, Kirchfeldstraß...","(51.213210849999996, 6.773057660453232, 0.0)",51.213211,6.773058,0.0
323,5,111,0,Fliedner Klinik,4,-,Martin-Luther-Platz 26,40212,Düsseldorf,"Martin-Luther-Platz 26, 40212, Düsseldorf, Ger...","(26, Martin-Luther-Platz, Stadtmitte, Stadtbez...","(51.2251325, 6.7817526, 0.0)",51.225133,6.781753,0.0
324,5,111,0,Johanniter-Tagesklinik,4,-,Marbacher Str. 79,40597,Düsseldorf,"Marbacher Str. 79, 40597, Düsseldorf, Germany","(Johanniter-Tagesklinik Düsseldorf-Benrath, 79...","(51.1643955, 6.8639209, 0.0)",51.164395,6.863921,0.0
325,5,111,0,Krankenanstalten,4,558,Kreuzbergstr. 79,40489,Düsseldorf,"Kreuzbergstr. 79, 40489, Düsseldorf, Germany","(Florence-Nightingale-Krankenhaus, 79, Kreuzbe...","(51.3003466, 6.7493957, 0.0)",51.300347,6.749396,0.0


In [19]:
df.latitude.isnull().sum()

4

In [24]:
df.dropna() 

,Land,Kreis,Gemeinde,Name1,Traeger,Betten_Ins,Straße/Hausnr,PLZ,Ort,address,location,point,latitude,longitude,altitude
321,5,111,0,Dominikus-Krankenhaus,4,200,Am Heerdter Krankenhaus 2,40549,Düsseldorf,"Am Heerdter Krankenhaus 2, 40549, Düsseldorf, ...","(Schön Klinik Düsseldorf, 2, Am Heerdter Krank...","(51.23135345, 6.727251670737527, 0.0)",51.231353,6.727252,0.0
322,5,111,0,Ev. Krankenhaus,4,568,Kirchfeldstr. 40,40217,Düsseldorf,"Kirchfeldstr. 40, 40217, Düsseldorf, Germany","(Evangelisches Krankenhaus, 40, Kirchfeldstraß...","(51.213210849999996, 6.773057660453232, 0.0)",51.213211,6.773058,0.0
323,5,111,0,Fliedner Klinik,4,-,Martin-Luther-Platz 26,40212,Düsseldorf,"Martin-Luther-Platz 26, 40212, Düsseldorf, Ger...","(26, Martin-Luther-Platz, Stadtmitte, Stadtbez...","(51.2251325, 6.7817526, 0.0)",51.225133,6.781753,0.0
324,5,111,0,Johanniter-Tagesklinik,4,-,Marbacher Str. 79,40597,Düsseldorf,"Marbacher Str. 79, 40597, Düsseldorf, Germany","(Johanniter-Tagesklinik Düsseldorf-Benrath, 79...","(51.1643955, 6.8639209, 0.0)",51.164395,6.863921,0.0
325,5,111,0,Krankenanstalten,4,558,Kreuzbergstr. 79,40489,Düsseldorf,"Kreuzbergstr. 79, 40489, Düsseldorf, Germany","(Florence-Nightingale-Krankenhaus, 79, Kreuzbe...","(51.3003466, 6.7493957, 0.0)",51.300347,6.749396,0.0
326,5,111,0,Krankenhaus Mörsenbroich-Rath,4,365,Amalienstr. 9,40472,Düsseldorf,"Amalienstr. 9, 40472, Düsseldorf, Germany","(Augusta Krankenhaus, 9, Amalienstraße, Rath, ...","(51.26669, 6.824603765389234, 0.0)",51.266690,6.824604,0.0
327,5,111,0,LVR-Klinikum Düsseldorf,1,475,Bergische Landstr. 2,40629,Düsseldorf,"Bergische Landstr. 2, 40629, Düsseldorf, Germany",(LVR-Klinikum Düsseldorf für Psychiatrie und P...,"(51.2439167, 6.8443747955819605, 0.0)",51.243917,6.844375,0.0
328,5,111,0,Marien-Hospital,4,439,Rochusstr. 2,40479,Düsseldorf,"Rochusstr. 2, 40479, Düsseldorf, Germany","(Marien-Hospital, 2, Rochusstraße, Pempelfort,...","(51.235961399999994, 6.785556772058095, 0.0)",51.235961,6.785557,0.0
329,5,111,0,Medical Center Düsseldorf,5,41,Hans-Günther-Sohl-Str. 6-10,40235,Düsseldorf,"Hans-Günther-Sohl-Str. 6-10, 40235, Düsseldorf...","(Luisenkrankenhaus, 6-10, Luise-Rainer-Straße,...","(51.2347233, 6.8208559, 0.0)",51.234723,6.820856,0.0
330,5,111,0,Paracelsus Klinik Golzheim,5,104,Friedrich-Lau-Str. 11,40474,Düsseldorf,"Friedrich-Lau-Str. 11, 40474, Düsseldorf, Germany","(Paracelsus-Klinik Golzheim, 11, Friedrich-Lau...","(51.254529899999994, 6.759477977071068, 0.0)",51.254530,6.759478,0.0


## Save points as pickle

In [25]:
df.to_pickle("Points_2015.pkl")